# Exploración Inicial

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [1]:
import pandas as pd
import pickle

import pandas as pd
import dask.dataframe as dd
import time

import scipy.stats
import re

## Obtencion de AGEBS de interés:

In [11]:
df_estaciones = pd.read_csv("./SIG/data/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv")
df_estaciones.head()

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
0,440,440 REAL DE MAYORAZGO-PUENTE XOCO,440 - Real De Mayorazgo-Puente Xoco,S/N,3330.0,1,Ampliación Granada,NaN,433,19.360650,...,NaN,NaN,NaN,"19.36065,-99.168669",0967,14,1,9,4550,Movement Zone 4550
1,283,283 AV. COYOACÁN-RAFAEL DONDE,283 - Av. Coyoacán-Rafael Donde,S/N,3103.0,1,Ampliación Granada,NaN,279,19.398270,...,NaN,NaN,NaN,"19.39827,-99.167383",0045,14,1,9,4551,Movement Zone 4551
2,279,279 LA MORENA-PROVIDENCIA,279 - La Morena-Providencia,S/N,3103.0,DVN,Del Valle Norte,NaN,164,19.399908,...,NaN,NaN,NaN,"19.399908,-99.166553",0045,14,1,9,4551,Movement Zone 4551
3,443,443 BRUNO TRAVEN-GOLONDRINAS,443 - Bruno Trave-Avenida México-Coyoacan,S/N,3340.0,1,Ampliación Granada,NaN,432,19.359655,...,NaN,NaN,NaN,"19.359655,-99.162025",0971,14,1,9,4553,Movement Zone 4553
4,432,432 CARRILLO PUERTO-AV. MÉXICO-COYOACÁN,432 - Carrillo Pto-Golondrinas,S/N,3340.0,GPM,General Pedro Maria Anaya,NaN,442,19.361296,...,NaN,NaN,NaN,"19.361296,-99.162581",0971,14,1,9,4553,Movement Zone 4553


In [14]:
df_estaciones['CVE_AGEB'] = [''.join(filter(lambda x: x.isdigit(), row)) for row in df_estaciones['CVE_AGEB']]

In [16]:
lista_agebs = df_estaciones['CVE_AGEB'].unique().tolist()
len(lista_agebs)

143

In [19]:
lista_agebs[:5]

[967, 45, 971, 219, 187]

Guardamos lo AGEBS en una lista:

In [20]:
with open("./data/lista_agebs.txt", "wb") as fp:
    pickle.dump(lista_agebs, fp)

## Leamos el archivo de viajes de UBER

In [61]:
path_uber_trips = "/Users/danielbustillos/Documents/ecobici_viz/data/uber/mexico_city-agebs-2019-2-OnlyWeekdays-HourlyAggregate.csv"

In [62]:
## cambiar a pandas

In [63]:
t=time.clock()
df_trips = dd.read_csv(path_uber_trips)
df_trips = df_trips.compute()
print("load train: " , time.clock()-t)

load train:  52.899135


In [64]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,3400,3458,12,2388.38,495.31,2342.07,1.21
1,4592,2596,23,2670.75,506.90,2619.95,1.22
2,4718,2885,6,957.27,158.18,944.96,1.17
3,4735,1406,18,3557.00,1044.99,3401.00,1.35
4,4670,2892,23,1082.94,475.12,1026.10,1.34


In [70]:
list(set(lista_agebs) - set(df_trips["sourceid"].unique()))

[1356, 717, 783, 1360, 49, 401, 533, 1337, 1341, 1214]

In [71]:
df_trips = df_trips[df_trips["sourceid"].isin(lista_agebs)]

In [72]:
#df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [73]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [74]:
df_trips.reset_index(drop=True, inplace=True)

In [75]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1534,3304,4,1448.04,446.76,1399.74,1.27
1,76,4671,10,2812.74,646.22,2746.81,1.24
2,875,1947,17,3026.17,427.75,2996.39,1.15
3,1464,4814,20,1645.67,313.17,1615.15,1.22
4,191,4616,11,3082.92,1786.94,2752.87,1.55


In [76]:
df_trips["sourceid"].nunique()

133

### tiempo máximo y mínimos por viaje y hora.
Suponemos una distribución normal con media 'mean_travel_time' y desviación estandar 'standard_deviation_travel_time>'

In [77]:
def travel_min(mean,std):
    min = scipy.stats.norm(mean, std).ppf(.10)
    #max = scipy.stats.norm(mean, std).ppf(.90)
    return(min)
def travel_max(mean,std):
    #min = scipy.stats.norm(mean, std).ppf(.10)
    max = scipy.stats.norm(mean, std).ppf(.90)
    return(max)

In [78]:
df_trips['min_travel_time'] = 0
df_trips['max_travel_time'] = 0

In [79]:
df_trips['min_travel_time'] =  df_trips.apply(
    lambda row : travel_min(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [80]:
df_trips['max_travel_time'] =  df_trips.apply(
    lambda row : travel_max(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [81]:
df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [82]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,min_travel_time,max_travel_time
0,1534,3304,4,1448.04,446.76,1399.74,1.27,875.494023,2020.585977
1,76,4671,10,2812.74,646.22,2746.81,1.24,1984.575747,3640.904253
2,875,1947,17,3026.17,427.75,2996.39,1.15,2477.986318,3574.353682
3,1464,4814,20,1645.67,313.17,1615.15,1.22,1244.326496,2047.013504
4,191,4616,11,3082.92,1786.94,2752.87,1.55,792.864245,5372.975755


In [83]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
min_travel_time                             float64
max_travel_time                             float64
dtype: object

In [84]:
df_viajes_ageb.groupby(['Ciclo_Estacion_Retiro','Ciclo_Estacion_Arribo','Hora_Retiro']).mean().reset_index(drop=False)

NameError: name 'df_viajes_ageb' is not defined